# Principal Component Analysis and Random Forest 

The goal of this notebook is to perform PCA and Random Forest to provide insight into our
full data. 

## Data Exploration

In [ ]:
# Define some packages and functions that will be used throughout the notebook

import pandas as pd
import numpy as np
import statistics
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt


def most_common(row):
    return max(set(row), key=row.count)

def get_seconds(row):
    return row.second

def sort_dt(row):
    return sorted(row)

def get_td_mean(row):
    td = 0
    if len(row) > 2:
        for i in range (0,len(row)-1):
            td += row[i+1]-row[i]
        return td/(len(row)-1)
    else:
        return 0
        
def get_td_sd(row):
    sd = 0
    new_list = []
    if len(row) > 2:
        for i in range (0,len(row)-1):
            new_list.append(row[i+1]-row[i])
        return statistics.stdev(new_list)
    else:
        return 0
        

In [ ]:
# import original data file from 23rd of May 2019

data = pd.read_csv("/Users/siboraseranaj/Downloads/stingar_full-20190523(2).csv")
data.head()

In [ ]:
data.columns

In [ ]:
data.dtypes

## Feature Engineering

First we engineered features based on the variables we already had. Most of our data was string data, so it was neccessary step. 
The features we created were:

    1. mean_time_difference which shows the mean difference in time between two consecutive          attacks from the same IP
    2. sd_time_difference which shows the standard deviation in time difference between two          consecutive attacks from the same ip
    3. length_username which shows the length of the username input by the attacker
    4. length_password which shows the length of the password input by the attacker
    5. sensor_number which shows the number of sensors hit by the attacker 

 

In [ ]:
data['d_time'] = pd.to_datetime(data['d_time']).values.astype(np.int64)

In [ ]:
# create feature mean_time_difference which shows the mean difference in time between two consecutive attacks from the same ip

data_grouped_td = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':lambda x: x.tolist()})
data_grouped_td['d_time'] = data_grouped_td['d_time'].apply(sort_dt)
data_grouped_td['d_time'] = data_grouped_td['d_time'].apply(get_td_mean)
data_grouped_td['d_time'] = pd.to_datetime(data_grouped_td['d_time'], unit='ns')
data_grouped_td['d_time'] = data_grouped_td['d_time'].apply(get_seconds)
data_grouped_td = data_grouped_td.rename({'d_time':'mean_time_difference'},axis = 1)
data_grouped_td.head()

In [ ]:
# create feature sd_time_difference which shows the standard deviation in time difference between two consecutive attacks from the same ip

data_grouped_sd = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':lambda x: x.tolist()})
data_grouped_sd['d_time'] = data_grouped_sd['d_time'].apply(sort_dt)
data_grouped_sd['d_time'] = data_grouped_sd['d_time'].apply(get_td_sd)
data_grouped_sd['d_time'] = pd.to_datetime(data_grouped_sd['d_time'], unit='ns')
data_grouped_sd['d_time'] = data_grouped_sd['d_time'].apply(get_seconds)
data_grouped_sd = data_grouped_sd.rename({'d_time':'sd_time_difference'},axis = 1)
data_grouped_sd.head() 

In [ ]:
data_grouped = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':np.mean})
data_grouped['d_time'] = pd.to_datetime(data_grouped['d_time'], unit='ns')
data_grouped.head()

In [ ]:
data_grouped1 = data[['src_ip','d_time']].groupby('src_ip',as_index = False).agg({'d_time':np.std})
data_grouped1['d_time'] = pd.to_datetime(data_grouped1['d_time'], unit='ns')
data_grouped1['d_time'] = data_grouped1['d_time'].apply(get_seconds)
data_grouped1.head()

In [ ]:
# Find the number of sensor hit by the attacker 
def rem_dups(array):
    myset = set(array)
    return len(list(myset))

data_grouped2 = data[['sensor','src_ip']].groupby('src_ip',as_index = False).agg({'sensor':lambda x: x.tolist()})
data_grouped2['sensor_number'] = data_grouped2['sensor'].apply(rem_dups)
data_grouped2['sensor_number'].value_counts()

In [ ]:
data_final = data_grouped.merge(data_grouped1, left_on='src_ip', right_on='src_ip')
data_final = data_final.merge(data_grouped_td,left_on='src_ip', right_on='src_ip')
data_final = data_final.merge(data_grouped_sd,left_on='src_ip', right_on='src_ip')
data_final = data_final.merge(data_grouped2,left_on='src_ip', right_on='src_ip')
data_final = data_final.rename({'d_time_x':'mean_time_of_attack','d_time_y':'sd_time_of_attack','sensor':'all_sensors'},axis = 1)

In [ ]:
data_final.head()

In [ ]:
data_final.drop(["mean_time_of_attack", "sd_time_of_attack"], axis = 1)

In [ ]:
data_final.drop(["mean_time_of_attack", "sd_time_of_attack", "all_sensors"], axis = 1, inplace= True)

In [ ]:
data_final.head()

In [ ]:
new = data[["ssh_username", "src_ip"]].dropna()

In [ ]:
new["length_username"] = new["ssh_username"].apply(len)

In [ ]:
user_length = new.groupby("src_ip").mean()

In [ ]:
data_final.head()

In [ ]:
current = pd.merge(user_length, data_final, how = "outer", on = "src_ip")

In [ ]:
current.head()

In [ ]:
current['length_username'].fillna(value = current['length_username'].mean(), inplace = True)

In [ ]:
current.head()

In [ ]:
new_command = data[['src_ip', 'command']]
new_command.dropna(inplace = True)
new_command['length_command'] = new_command['command'].apply(len) 

In [ ]:
feature = new_command.groupby('src_ip').mean()

In [ ]:
current = pd.merge(current, feature, how = 'outer', on = "src_ip")

In [ ]:
current.head()

In [ ]:
current['length_command'].fillna(value = current['length_command'].mean(), inplace = True)

In [ ]:
current.head()

In [ ]:
data['app'].value_counts()

In [ ]:
counts = data['app'].value_counts()
res = data[~data['app'].isin(counts[counts < 27].index)]
res['app'].value_counts()
honeypot = res[['app', 'src_ip']]
honeypot.head()

In [ ]:
current.head()

In [ ]:
honeypot['app'].value_counts()
honeypot.groupby('src_ip')

In [ ]:
new_current = pd.merge(current, honeypot, how = 'inner', on = 'src_ip')
new_current.head()

In [ ]:
current

In [ ]:
dat = data.groupby('src_ip')[['src_ip', 'app']].head()
temp = dat.drop_duplicates()

In [ ]:
let = pd.get_dummies(temp['app'])
let.head()

In [ ]:
det = pd.concat([let,temp], axis = 1)
det['app'].value_counts()
counts = det['app'].value_counts()
det = det[~det['app'].isin(counts[counts < 100].index)]
det['app'].value_counts()

In [ ]:
current.columns

In [ ]:
current.drop_duplicates('src_ip', inplace = True)

In [ ]:
current

In [ ]:
head = data.groupby('src_ip').count()
head.head()
head.reset_index(inplace = True)
val = head[['src_ip', 'app']]
val.head()

In [ ]:
current = pd.merge(current, val, how = 'inner', on = 'src_ip')

In [ ]:
current.rename(columns={'app':'daily_frequency'}, inplace=True)

In [ ]:
current

In [ ]:
new = data[['src_ip', 'ssh_password']]
new.head()

In [ ]:
new_data = data[['src_ip', 'ssh_password']]
new_data.head()
new_data.dropna(inplace = True)

In [ ]:
new_data['length_password'] = new_data['ssh_password'].apply(len)

In [ ]:
new_data.drop(['ssh_password'], axis = 1, inplace = True)

In [ ]:
new_data.drop_duplicates(inplace= True)

In [ ]:
new_data.head()

In [ ]:
current = pd.merge(current, new_data, how = 'outer', on = 'src_ip')
current['length_password'].fillna(value = current['length_password'].mean(), inplace = True)

In [ ]:
current.drop_duplicates(inplace= True)
current.drop_duplicates('src_ip', inplace= True)
current.head()

In [ ]:
dat_t = data.groupby('src_ip')['dest_port'].nunique()
new_dat_t = dat_t.reset_index()
new_dat_t.head()

In [ ]:
current = pd.merge(new_dat_t, current, how = 'inner', on = 'src_ip')

In [ ]:
current['dest_port'].value_counts()

In [ ]:
current.rename(columns={'dest_port':'dest_port_number'}, inplace=True)

In [ ]:
#current['length_command'].value_counts()

In [ ]:
#current['length_password'].value_counts()

In [ ]:
current['length_username'].value_counts()

## PCA - all honeypots

This was my first attempt on Principal Component Analysis. PCA is a statistical procedure that uses orthogonal transformation to convert many possibly correlated numerical variables into a specified number of linearly uncorrelated variables. These variables are called principal components. In this analysis I perform PCA on data from all the honeypots, and my target is signature.

In [ ]:
# Import neccessary libraries for PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# We need to create a target in order to perform PCA
# Find most common signature occuring for each ip
signature = data[['src_ip', 'signature']]
signature_grouped = signature.groupby('src_ip', as_index = False).agg({'signature':lambda x: x.tolist()})
signature_grouped['signature'] = signature_grouped['signature'].apply(most_common)
signature_grouped.head()

In [ ]:
# Create new column named target which is basically a code for each signature
# (PCA only accepts numerical values)

signature_grouped['signature'].unique()
def target(row):
    if row == 'Connection to Honeypot':
        return 1
    elif row == 'SSH login attempted on cowrie honeypot':
        return 2
    elif row == 'SSH session on cowrie honeypot':
        return 3
    elif row == 'command attempted on cowrie honeypot':
        return 4
    else:
        return 5
signature_grouped['target'] = signature_grouped['signature'].apply(target) 
signature_grouped.head()

In [ ]:
current = pd.merge(signature_grouped, current, how = 'inner', on = 'src_ip')

In [ ]:
current = current.drop(['signature'], axis = 1)
current['target'].value_counts()

In [ ]:
# scale your data
scaler = StandardScaler()
current_pca_all = current.drop(["src_ip"], axis = 1)
scaler.fit(current_pca_all)

In [ ]:
scaled_data = scaler.transform(current_pca_all)

In [ ]:
# Specify your dimensions
pca = PCA(n_components=7)

In [ ]:
# Fit the scaled data
pca.fit(scaled_data)

In [ ]:
x_pca = pca.transform(scaled_data)

In [ ]:
scaled_data.shape

In [ ]:
x_pca.shape

In [ ]:
# Plot your components against each other

plt.figure(figsize=(9,7))
plt.scatter(x_pca[:,0],x_pca[:,1],c=current['target'], cmap='plasma')
plt.title('Principal Component Analysis', fontsize = 15)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
#plt.xlim(-1.5, 2.5)
#plt.ylim(-5, 5)

In [ ]:
# This is neccessary to view the correlation of individual components to one another
pca.components_
sns.heatmap(pca.components_)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

## PCA - Dionaea honeypot

We realised that performing PCA on data from all the honeypots was not ideal because some of the data gathered only apply to a specific honeypot and no to the other. Therefore, we focused on dionaea honeypot and identifing components there. To do that, we first found the most common honeypot for each ip, and the target used was again signature. Interestingly enough, most of the signatures are of the same kind, and there is a distinct cluster forming.

In [ ]:
# find most common honeypot per ip
app = data[['src_ip', 'app']]
app_grouped = app.groupby('src_ip', as_index = False).agg({'app':lambda x: x.tolist()})
app_grouped['app'] = app_grouped['app'].apply(most_common)
app_grouped.head()
app_grouped['app'].value_counts()

In [ ]:
dionaea_grouped = app_grouped.loc[app_grouped['app'] == "dionaea"]

In [ ]:
dionaea_grouped.head()

In [ ]:
current_dionaea = pd.merge(dionaea_grouped, current, how = 'inner', on = 'src_ip')
current_dionaea.head()

In [ ]:
scaler = StandardScaler()
current_pca_dionaea = current_dionaea.drop(["src_ip", "app"], axis = 1)
current_pca_dionaea.head()
scaler.fit(current_pca_dionaea)

In [ ]:
scaled_data = scaler.transform(current_pca_dionaea)

In [ ]:
pca = PCA(n_components=7)

In [ ]:
pca.fit(scaled_data)

In [ ]:
x_pca = pca.transform(scaled_data)

In [ ]:
scaled_data.shape

In [ ]:
x_pca.shape

In [ ]:
plt.figure(figsize=(9,7))
plt.scatter(x_pca[:,0],x_pca[:,1],c=current_dionaea['target'], cmap='plasma')
plt.title('Principal Component Analysis', fontsize = 15)

plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.xlim(-1.5, 5)
plt.ylim(-2, 2)

In [ ]:
pca.components_
sns.heatmap(pca.components_)

In [ ]:
sum(pca.explained_variance_ratio_[0:7])

In [ ]:
pca.components_

## Random Forest - random danger data

In [ ]:
# Import neccessary Random Forest libabries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

In [ ]:

import random
rf_data = current.copy()
rf_data = rf_data[['src_ip','mean_time_difference','sd_time_difference','sensor_number','length_password']]

#adding new column (random data) which should be our output. Potentionally we will have this column
rf_data['danger'] = random.choices(population = [1,0],weights = [0.75,0.25], k = len(rf_data))
train = rf_data[:8000]
test = rf_data[8000:]



In [ ]:
predictor_vars = ['mean_time_difference','sd_time_difference','sensor_number','length_password']
X, y = train[predictor_vars], train.danger

In [ ]:
modelRandom = RandomForestClassifier(max_depth=25)
modelRandomCV = cross_val_score(modelRandom,X,y,cv = 5)

In [ ]:
modelRandomCV

In [ ]:
modelRandom = RandomForestClassifier(max_depth=25)
modelRandom.fit(X,y)

In [ ]:
predictions = modelRandom.predict(test[predictor_vars])
predictions

In [ ]:
test['predictions'] = predictions
test.head()

In [ ]:
def check(row):
    if row['danger'] == row['predictions']:
        return 'Correct'
    else:
        return 'Incorrect'
test['check'] = test.apply(check,axis = 1)

accuracy_total = len(test[test['check'] == 'Correct'])/len(test)
accuracy_total
#Hard to predict random data. Hence, low accuracy.

In [ ]:
danger = test[test['danger'] == 1]
danger_accuracy = len(danger[danger['check'] == 'Correct'])/len(danger)
danger_accuracy
#As we said, since data are random it's hard to predict outcomes. Especially for "danger"(since "danger" label is only about 25% of our training data)

In [ ]:
# Visualise classical Confusion Matrix
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(test.danger, predictions)
print(CM)

# Visualize it as a heatmap
import seaborn
seaborn.heatmap(CM)
plt.show()

In [ ]:
# from sklearn.metrics import roc_curve, auc
# fpr, tpr, _ = roc_curve(test.danger,predictions)

# plt.clf()
# plt.plot(fpr, tpr)

# plt.xlim([-0.05, 1.05])
# plt.ylim([-0.05, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()

## Random Forest - tags

In [ ]:
values = {"tags": "cloud"}
data.fillna(values, inplace = True)
data['tags'].value_counts()

In [ ]:
def tags(row):
    if row == 'localnet,durham,honeynet' or row =='localnet,durham,sciencedmz' or row == 'localnet,durham':
        return 0
    if row == 'cloud':
        return 1

In [ ]:
data['new_tags'] = data['tags'].apply(tags) 
data['new_tags'].value_counts()

new_data = data[['src_ip', 'new_tags']]

In [ ]:
rftags = pd.merge(current, new_data, how = "inner", on = "src_ip")
rftags.drop_duplicates(subset = 'src_ip', inplace = True)
rftags['new_tags'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X = rftags[['mean_time_difference','sd_time_difference','sensor_number','length_password', 'length_username', "length_command", 'daily_frequency']]
y = rftags['new_tags']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [ ]:
modelTags = RandomForestClassifier(max_depth = 25)
modelTags.fit(X_train, y_train)

In [ ]:
predictions_tags = modelTags.predict(X_test)
predictions_tags
modelTags.feature_importances_

In [ ]:
# Visualise classical Confusion Matrix
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_test, predictions_tags)
print(CM)

# Visualize it as a heatmap
import seaborn
seaborn.heatmap(CM)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions_tags))